# SubspaceSlicePlotter Demo

`SubspaceSlicePlotter` visualises any `Subset` (polyhedral or not) by slicing it on a 1D, 2D, or 3D affine subspace. The interface is uniform: you supply a subset, a subspace, and call `.plot()`.

This notebook shows the two main cases:
1. **Polyhedral sets** — exact, vertex-based rendering via halfspace intersection.
2. **Non-polyhedral sets** (Ball, Ellipsoid) — raster membership-oracle sampling.


## 1. Imports


In [ ]:
%matplotlib inline

import sys
import warnings
sys.path.insert(0, "..")

import numpy as np
import matplotlib.pyplot as plt

from pygeoinf.hilbert_space import EuclideanSpace
from pygeoinf.subsets import HalfSpace, PolyhedralSet, Ball, Ellipsoid
from pygeoinf.subspaces import AffineSubspace
from pygeoinf.visualization import SubspaceSlicePlotter

# Suppress the solver warning — we only need geometric slicing, not Bayesian conditioning
warnings.filterwarnings("ignore", message="Constructing a subspace from a tangent basis without a solver")


## 2. Polyhedral sets — exact rendering

We work in **R³**.  All polyhedral examples use `PolyhedralSet`, which triggers the fast exact path (Chebyshev centre → halfspace intersection → convex hull) instead of grid sampling.


### 2a. Box — 2D slice (the xy-plane through the origin)


In [ ]:
# --- Space and box definition ---
R3 = EuclideanSpace(3)

# Basis vectors as floats (required by the in-place scaling in EuclideanSpace)
e1 = np.array([1., 0., 0.])
e2 = np.array([0., 1., 0.])
e3 = np.array([0., 0., 1.])

# Box  [-1, 1]^3  defined by 6 halfspaces: ±e_i . x <= 1
box = PolyhedralSet(R3, [
    HalfSpace(R3,  e1,  1.0),   # x <= 1
    HalfSpace(R3, -e1,  1.0),   # -x <= 1  (x >= -1)
    HalfSpace(R3,  e2,  1.0),
    HalfSpace(R3, -e2,  1.0),
    HalfSpace(R3,  e3,  1.0),
    HalfSpace(R3, -e3,  1.0),
])

# --- 2D slice: the xy-plane (z = 0) ---
xy_plane = AffineSubspace.from_tangent_basis(R3, [e1, e2])

# --- Three lines of user code ---
plotter = SubspaceSlicePlotter(box, xy_plane)
fig, ax, verts = plotter.plot(bounds=(-1.5, 1.5, -1.5, 1.5), show_plot=False)
ax.set_title("Box  [-1,1]³ — 2D slice (z = 0)")
plt.show()
print("Polygon vertices:\n", verts)


Polygon vertices:
 [[-1. -1.]
 [ 1. -1.]
 [ 1.  1.]
 [-1.  1.]]


/home/adrian/PhD/Inferences/pygeoinf/convex_analysis_demos/../pygeoinf/visualization.py:974: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_10847/931146965.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 2b. Box — 1D slice (a diagonal line)


In [ ]:
# 1D slice along the diagonal direction (1,1,1)/sqrt(3), shifted into the interior
diag = np.array([1., 1., 1.]) / np.sqrt(3.)
x_offset = 0.1 * e1 + 0.1 * e2   # translate so the line passes through a visible region
diag_line = AffineSubspace.from_tangent_basis(R3, [diag], translation=x_offset)

plotter = SubspaceSlicePlotter(box, diag_line)
fig, ax, interval = plotter.plot(bounds=(-2.0, 2.0), show_plot=False)
ax.set_title("Box  [-1,1]³ — 1D slice along diagonal (1,1,1)/√3")
plt.show()
print(f"Interval endpoints: [{interval[0]:.4f}, {interval[1]:.4f}]")


Interval endpoints: [-1.7321, 1.5588]


/tmp/ipykernel_10847/578759821.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 2c. Simplex — 2D cross-section at z = 0.2

Slicing the standard simplex with the plane $z = 0.2$ gives a smaller triangle.  
We pass a `translation=0.2*e3` to shift the xy-plane to that level.


In [ ]:
# Standard simplex: x>=0, y>=0, z>=0, x+y+z<=1
ones = np.array([1., 1., 1.])
simplex = PolyhedralSet(R3, [
    HalfSpace(R3, -e1, 0.0),   # x >= 0  (i.e. -x <= 0)
    HalfSpace(R3, -e2, 0.0),   # y >= 0
    HalfSpace(R3, -e3, 0.0),   # z >= 0
    HalfSpace(R3,  ones, 1.0), # x+y+z <= 1
])

# Cross-section at z = 0.2: use xy-plane shifted by 0.2*e3
z02_plane = AffineSubspace.from_tangent_basis(R3, [e1, e2], translation=0.2 * e3)

plotter = SubspaceSlicePlotter(simplex, z02_plane)
fig, ax, verts = plotter.plot(bounds=(-0.1, 1.0, -0.1, 1.0), show_plot=False)
ax.set_title("Standard simplex — 2D cross-section at z = 0.2")
plt.show()
print("Triangle vertices:\n", verts)


Triangle vertices:
 [[-2.77555756e-17 -2.77555756e-17]
 [ 8.00000000e-01  0.00000000e+00]
 [ 2.77555756e-17  8.00000000e-01]]


/tmp/ipykernel_10847/4219737146.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 3. Non-polyhedral sets — raster sampling

For non-polyhedral sets (Ball, Ellipsoid, …) the plotter falls back to membership-oracle grid sampling. The interface is identical.


### 3a. Unit ball — 2D slice


In [ ]:
center = R3.zero      # origin
ball = Ball(R3, center, radius=1.0)

# Same xy-plane as before
plotter = SubspaceSlicePlotter(ball, xy_plane, grid_size=100)
fig, ax, mask = plotter.plot(bounds=(-1.5, 1.5, -1.5, 1.5), cmap="Oranges", show_plot=False)
ax.set_title("Unit ball — 2D slice (z = 0)")
plt.show()


/tmp/ipykernel_10847/1212732815.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 3b. Unit ball — 1D slice


In [ ]:
x_axis_line = AffineSubspace.from_tangent_basis(R3, [e1])

plotter = SubspaceSlicePlotter(ball, x_axis_line, grid_size=200)
fig, ax, mask = plotter.plot(bounds=(-1.5, 1.5), color="tomato", show_plot=False)
ax.set_title("Unit ball — 1D slice (x-axis)")
plt.show()


/tmp/ipykernel_10847/2338813433.py:7: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### 3c. Ellipsoid — 2D slice (oblique plane)


In [ ]:
from pygeoinf.linear_operators import LinearOperator

# Ellipsoid with semi-axes (1, 0.5, 2): A = diag(1, 4, 0.25)
A_mat = np.diag([1.0, 4.0, 0.25])
A_op  = LinearOperator.from_matrix(R3, R3, A_mat)

ellipsoid = Ellipsoid(R3, R3.zero, radius=1.0, operator=A_op)

# Oblique plane spanned by e1 and e3
e1_e3_plane = AffineSubspace.from_tangent_basis(R3, [e1, e3])

plotter = SubspaceSlicePlotter(ellipsoid, e1_e3_plane, grid_size=120)
fig, ax, mask = plotter.plot(bounds=(-1.5, 1.5, -2.5, 2.5), cmap="Greens", show_plot=False)
ax.set_title("Ellipsoid (semi-axes 1, 0.5, 2) — 2D slice (y = 0 plane)")
plt.show()


/tmp/ipykernel_10847/2645266802.py:16: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 4. Customisation options

`SubspaceSlicePlotter` accepts several optional arguments to tweak appearance. All customisation is optional — the defaults are intentionally clean.

| Argument | Effect |
|---|---|
| `cmap` | Colormap for 2D filled region |
| `color` | Bar color for 1D |
| `alpha` | Transparency (0–1) |
| `grid_size` | Raster resolution (non-polyhedral only) |
| `bounds` | Plot window |
| `ax` | Inject into an existing `Axes` |


In [ ]:
# Side-by-side: box (exact) vs ball (raster) — same plane, different colours
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

SubspaceSlicePlotter(box,  xy_plane, alpha=0.4).plot(
    bounds=(-1.5, 1.5, -1.5, 1.5), cmap="Blues",   show_plot=False, ax=axes[0])
axes[0].set_title("Box (exact, α=0.4)")

SubspaceSlicePlotter(ball, xy_plane, alpha=0.7, grid_size=80).plot(
    bounds=(-1.5, 1.5, -1.5, 1.5), cmap="Reds",    show_plot=False, ax=axes[1])
axes[1].set_title("Ball (raster, α=0.7)")

plt.tight_layout()
plt.show()


/tmp/ipykernel_10847/256013470.py:14: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
